In [ ]:
#数据获取与处理
import pandas as pd
import numpy as np
zero_flow = pd.read_excel('C:/Users/690/Desktop/每周五品牌星球运营数据需求/【星球二期-秀秀tab3点击】-【星球二期-秀秀tab3点击】数据导出.xlsx')
zero_flow['秀秀首页曝光量(求和)'] = zero_flow['秀秀首页曝光量(求和)'].map(str)
zero_flow['秀秀首页曝光uv(求和)'] = zero_flow['秀秀首页曝光uv(求和)'].map(str)
zero_flow['0元购tab点击量(求和)'] = zero_flow['0元购tab点击量(求和)'].map(str)
zero_flow['0元购tab点击uv(求和)'] = zero_flow['0元购tab点击uv(求和)'].map(str)
title = tuple(zero_flow.columns.tolist())
tuples = [tuple(x) for x in zero_flow.head(30).values]
tuples.insert(0,title)

#导入数据平台展示所需包
import remi.gui as gui
from remi import start, App

import io
import time
import threading
import random

from matplotlib.figure import Figure
from matplotlib.backends.backend_agg import FigureCanvasAgg
from matplotlib.pyplot import MultipleLocator
import matplotlib.pyplot as plt


#设置远程访问
# class RemoteLabel(gui.Label):
#     def __init__(self, text, **kwargs):
#         super(RemoteLabel, self).__init__(text, **kwargs)

#     # api function
#     def api_set_text(self, value1, value2):
#         self.set_text('parameters: %s - %s' % (value1, value2))
#         headers = {'Content-type': 'text/plain'}
#         return ['ok', headers]

#定义绘图类
class MatplotImage(gui.Image):
    ax = None

    def __init__(self, **kwargs):
        super(MatplotImage, self).__init__("/%s/get_image_data?update_index=0" % id(self), **kwargs)
        self._buf = None
        self._buflock = threading.Lock()

        self._fig = Figure(figsize=(10, 10))
        self.ax = self._fig.add_subplot(111)
        
        self.redraw()

    def redraw(self):
        canv = FigureCanvasAgg(self._fig)
        buf = io.BytesIO()
        canv.print_figure(buf, format='png')
        with self._buflock:
            if self._buf is not None:
                self._buf.close()
            self._buf = buf

        i = int(time.time() * 1e6)
        self.attributes['src'] = "/%s/get_image_data?update_index=%d" % (id(self), i)

        super(MatplotImage, self).redraw()

    def get_image_data(self, update_index):
        with self._buflock:
            if self._buf is None:
                return None
            self._buf.seek(0)
            data = self._buf.read()

        return [data, {'Content-type': 'image/png'}]


#网页元素添加
class MyApp(App):
    def init(self, *args):
        super(MyApp, self).init(*args)
    def main(self):
        
#         #设置远程访问
#         self.lbl = RemoteLabel('type in other page url "http://172.18.68.200:8081/label/api_set_text?value1=text1&value2=text2" !',
#         width='80%', height='50%', attributes={'id': 'label'})
#         self.lbl.style['margin'] = 'auto'
        
        #引入在线Bootstrap的css文件
        my_css_head = """
        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
        """
        #引入在线Bootstrap的js文件
        my_js_head = """
        <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js"></script>
        """

        self.page.children['head'].add_child('mycss', my_css_head)
        self.page.children['head'].add_child('myjs', my_js_head)
        
        #页面布局
        verticalContainer = gui.Container(width='100%',height='100%',margin='0px auto', style={'display': 'block', 'overflow': 'hidden'})
        horizontalContainer = gui.Container(width='100%',height='100%',layout_orientation=gui.Container.LAYOUT_HORIZONTAL, margin='0px', style={'display': 'block', 'overflow': 'auto', 'text-align': 'center'})
        subContainerLeft = gui.Container(width='50%',height='100%',style={'display': 'block', 'overflow': 'auto', 'text-align': 'center'})
#         subContainerLeft_1 = gui.Container(width='100%',height='10%',style={'display': 'block', 'overflow': 'auto', 'text-align': 'center'})
#         subContainerLeft_2 = gui.Container(width='100%',height='90%',style={'display': 'block', 'overflow': 'auto', 'text-align': 'center'})
        subContainerRight = gui.Container(style={'width':'50%','height':'100%','display': 'block', 'overflow': 'auto', 'text-align': 'center'})
        
        #Table=表格，第一个是表头
        myList_1 = tuples
        #表格设置，内容，大小
        self.tbl_1 = gui.Table.new_from_list(content=myList_1,width='630px',height='100%',margin='10px') #
        #样式选择
        self.tbl_1.add_class("table table-striped")
#         myList_2 = tuples[1:]
#         self.tbl_2 = gui.Table.new_from_list(content=myList_2,width='630px',height='100%',margin='10px')

        #下载表格
        fdownloader = gui.FileDownloader('download table', 
                                         'C:/Users/690/Desktop/每周五品牌星球运营数据需求/【星球二期-秀秀tab3点击】-【星球二期-秀秀tab3点击】数据导出.xlsx', 
                                         width=200, height=30, margin='10px')
        subContainerLeft.append(fdownloader, key='file_downloader')
        
        #输入框搜索
        self.tf = gui.TextInput(hint='Your Search')
        self.tf.add_class("form-control input-lg") #样式选择
        
        #下拉选择框进行渠道选择
        self.dd = gui.DropDown(width='200px')
        #字体大小
        self.dd.style.update({'font-size':'large'})
        #样式选择
        self.dd.add_class("form-control dropdown")
        #下拉框内容选择
        self.item1 = gui.DropDownItem("First Choice")
        self.item2 = gui.DropDownItem("Second Item")
        #定义self的功能，目前未定义
        self.dd.append(self.item1,'item1')
        self.dd.append(self.item2,'item2')

        
        #绘制图形
        plot_x = pd.to_datetime(zero_flow['date(按日)']).tolist()
        plot_y = zero_flow['0元购tab点击量(求和)'].map(int).tolist()
        #获取横纵坐标轴值
        self.plot_data_x = plot_x
        self.plot_data_y = plot_y
        #设置横纵坐标轴刻度
        self.mpl = MatplotImage(width='100%', height='80%')
#         self.mpl.style['margin'] = '10px'
        self.mpl.ax.set_title("test")
        self.mpl.ax.plot(self.plot_data_x,self.plot_data_y)
        #旋转横坐标轴标签角度
        """"for tick in self.mpl.ax.get_xticklabels():
            tick.set_rotation(45)"""
        self.mpl.redraw()
    
        #文本框
        text = gui.TextInput(width='100%',height='10%')
        text.set_text('显示数据洞察描述......')
        #self.txt.onchange.do(self.on_text_area_change)
        
        #标签
#         self.label_1 = gui.Label('This is a label')
#         self.label_1.style['background-color'] = 'lightblue'
#         self.label_1.style['height'] = '1080px'
#         self.label_2 = gui.Label('This is a label')
#         self.label_2.style['background-color'] = 'lightblue'
        
        
        #设置按钮
#         bt_consumption = gui.Button("consumption", style={'margin': '3px', 'background-color': 'orange'})
#         #bt_consumption.onclick.do(self.on_consumption_a_label_pressed)
#         bt_cost = gui.Button("cost", style={'margin': '3px', 'background-color': 'red'})
#         #bt_cost.onclick.do(self.on_cost_pressed)
#         bt_traffic = gui.Button("traffic", style={'margin': '3px', 'background-color': 'green'})
#         #bt_cost.onclick.do(self.on_cost_pressed)
        
        #设置菜单栏
        menu = gui.Menu(width='100%', height='30px')
        m1 = gui.MenuItem('广告投放', width=100, height=30)
        m2 = gui.MenuItem('其他', width=100, height=30)
        m11 = gui.MenuItem('流量', width=100, height=30)
#         m11.onclick.do(self.menu_open_clicked)
        m12 = gui.MenuItem('消耗', width=100, height=30)
#         m12.onclick.do(self.menu_open_clicked)
        m13 = gui.MenuItem('成本', width=100, height=30)
#         m13.onclick.do(self.menu_open_clicked)
        m111 = gui.MenuItem('年', width=50, height=20)
        m112 = gui.MenuItem('月', width=50, height=20)
        m113 = gui.MenuItem('周', width=50, height=20)
        m114 = gui.MenuItem('日', width=50, height=20)
        
        m121 = gui.MenuItem('年', width=50, height=20)
        m122 = gui.MenuItem('月', width=50, height=20)
        m123 = gui.MenuItem('周', width=50, height=20)
        m124 = gui.MenuItem('日', width=50, height=20)
        
        m131 = gui.MenuItem('年', width=50, height=20)
        m132 = gui.MenuItem('月', width=50, height=20)
        m133 = gui.MenuItem('周', width=50, height=20)
        m134 = gui.MenuItem('日', width=50, height=20)
        

        menu.append([m1,m2])
        m1.append([m11,m12,m13])
        m11.append([m111,m112,m113,m114])
        m12.append([m121,m122,m123,m124])
        m13.append([m131,m132,m133,m134])
        
        menubar = gui.MenuBar(width='100%', height='30px')
        menubar.append(menu)
        
        #设置日期控件
        self.ddate_start = gui.Date(width=200, height=20)
        self.ddate_start.set_value('2000-01-01')
#         self.date.onchange.do(self.date_changed)
        self.ddate_end = gui.Date(width=200, height=20)
        self.ddate_end.set_value('2000-01-31')

        #Build up the gui，主窗口挂载下面项目
        
        subContainerLeft.append([self.tf,self.dd,self.tbl_1])
#         subContainerLeft_2.append([self.tbl_2])
#         subContainerLeft.append([subContainerLeft_1,subContainerLeft_2])
        subContainerRight.append([self.ddate_start,self.ddate_end,self.mpl,text])
        horizontalContainer.append([subContainerLeft, subContainerRight])
        verticalContainer.append([menubar,horizontalContainer])
               
        return verticalContainer
# if __name__ == "__main__":
# starts the webserver
start(MyApp,address='0.0.0.0',port=8081,multiple_instance=True)


remi.server      INFO     Started httpserver http://0.0.0.0:8081/
remi.request     INFO     built UI (path=/)
127.0.0.1 - - [23/Jul/2020 19:42:16] "GET / HTTP/1.1" 200 -
remi.server.ws   INFO     connection established: ('127.0.0.1', 56632)
127.0.0.1 - - [23/Jul/2020 19:42:16] "GET /2145042960328/get_image_data?update_index=1595504536343349 HTTP/1.1" 200 -
remi.server.ws   INFO     handshake complete
127.0.0.1 - - [23/Jul/2020 19:42:40] "GET /2145044279240/download HTTP/1.1" 200 -
remi.request     ERROR    error processing GET request
Traceback (most recent call last):
  File "E:\Anaconda\lib\site-packages\remi\server.py", line 596, in do_GET
    self._process_all(path)
  File "E:\Anaconda\lib\site-packages\remi\server.py", line 675, in _process_all
    self.send_header(k, headers[k])
  File "E:\Anaconda\lib\http\server.py", line 516, in send_header
    ("%s: %s\r\n" % (keyword, value)).encode('latin-1', 'strict'))
UnicodeEncodeError: 'latin-1' codec can't encode characters in position